# Preprocess Data

Remember to run fetch_data.sh first to download other required github data repos!

## Imports

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta
from dateutil.parser import parse
import matplotlib.pyplot as plt
import math
import os
import pickle
import requests

## Helper functions

In [ ]:
def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False
    
def get_county(string):
    str_array = string.split(',')
    if len(str_array) < 3:
        return ''
    else:
        return str_array[0]

## Check that reference data directory exists

In [ ]:
# Check that the reference directory exists
for dirname, _, filenames in os.walk('../COVID-19/csse_covid_19_data/csse_covid_19_time_series/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Read covid 19 and census data

In [ ]:
# Covid confirmed cases and deaths
us_confirmed_df = pd.read_csv('../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
us_deaths_df = pd.read_csv('../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
world_confirmed_df = pd.read_csv('../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
world_deaths_df = pd.read_csv('../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

# US County level census data
# From https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv
county_census_df = pd.read_csv('./data/co-est2019-alldata.csv', encoding='latin-1')
county_census_df.head()

# World population data (for MOST countries)
world_population_df = pd.read_csv('./data/country_profile_variables.csv')
world_population_df = world_population_df[['country', 'Population in thousands (2017)']]
world_population_df = world_population_df.rename(columns={'Population in thousands (2017)': 'Population'})
# Add some missing countries
world_population_df.loc[len(world_population_df)] = ['Taiwan*', 23780]
world_population_df['Population'] *= 1000

## Combine US County COVID and Census Data

The output will be written into your ./data directory as `us_combined_df.pkl`. If this file already exists, the following cell will do an incremental update on the most recent missing dates.

If you suspect having corrupted data, you can delete this file, and the following cell will recreate the dataframe from scratch.

In [ ]:
# Load combined dataframe if it already exists, and select date to do incremental updates.

# Output dataframe columns
column_names = ['FIPS', 'County', 'Province_State', 'Country_Region', 'Date', 'Cases', 'Deaths', 'Population']

if os.path.exists('./data/us_combined_df.pkl'):
    with open('./data/us_combined_df.pkl', 'rb') as f:
        us_combined_df = pickle.load(f)
else:
    us_combined_df = pd.DataFrame(columns=column_names)

if len(us_combined_df):
    start_date = us_combined_df['Date'].max() + timedelta(0, 0, 1)
else:
    start_date = datetime(2019, 1, 1)

# Use date conversions to extract correct dates from column names in COVID dataset
date_cols = [x for x in list(us_confirmed_df) if is_date(x)]
dates = [datetime.strptime(x , '%m/%d/%y') for x in date_cols]
dates = [x for x in dates if x >= start_date]
date_cols = [x for x in date_cols if datetime.strptime(x, '%m/%d/%y') >= start_date]

# Append rows that have confirmed cases, deaths, and populations included.
for index, row in us_confirmed_df.iterrows():
    fips = row['FIPS']
    county = get_county(row['Combined_Key'])
    if math.isnan(fips):
        print('skipping county', county, fips, population)
        continue
    population = county_census_df[
        (county_census_df.STATE == int(fips / 1000))
        & (county_census_df.COUNTY == int(fips % 1000))]['POPESTIMATE2019']
    if len(population) != 1:
        print('skipping county', county, fips, population)
        continue
    population = population.to_numpy()[0]
    for (date_col, date) in zip(date_cols, dates):
        confirmed = row[date_col]
        if confirmed == 0:
            continue
        if date_col in us_deaths_df:
            deaths = us_deaths_df[us_deaths_df.FIPS == row['FIPS']][date_col].to_numpy()[0]
        else:
            deaths = 0
            
        values = [fips, county, row['Province_State'], row['Country_Region'], date, confirmed, deaths, population]
        df_length = len(us_combined_df)
        us_combined_df.loc[df_length] = values
    if index % 100 == 0:
        print('processed {} out of {}'.format(index, len(us_confirmed_df)))
us_combined_df = us_combined_df.drop_duplicates(['Date', 'FIPS'], keep='last') # Drop duplicates just in case
us_combined_df.tail() # Check output

In [ ]:
# Write to disk
with open('./data/us_combined_df.pkl', 'wb') as f:
    pickle.dump(us_combined_df, f)

## Combine worldwide country population and COVID-19 data

The output will be written into your ./data directory as `world_combined_df.pkl`. If this file already exists, the following cell will do an incremental update on the most recent missing dates.

If you suspect having corrupted data, you can delete this file, and the following cell will recreate the dataframe from scratch.

In [ ]:
column_names = ['Country_Region', 'Date', 'Cases', 'Deaths', 'Population']

if os.path.exists('./data/world_combined_df.pkl'):
    with open('./data/world_combined_df.pkl', 'rb') as f:
        world_combined_df = pickle.load(f)
else:
    world_combined_df = pd.DataFrame(columns=column_names)

if len(world_combined_df):
    start_date = world_combined_df['Date'].max() + timedelta(0, 0, 1)
else:
    start_date = datetime(2019, 1, 1)

# Use date conversions to extract correct dates from column names in COVID dataset
date_cols = [x for x in list(us_confirmed_df) if is_date(x)]
dates = [datetime.strptime(x , '%m/%d/%y') for x in date_cols]
dates = [x for x in dates if x >= start_date]
date_cols = [x for x in date_cols if datetime.strptime(x, '%m/%d/%y') >= start_date]

# Aggregate regions for countries
world_confirmed_agg_df = world_confirmed_df.groupby(['Country/Region']).agg({
    d: 'sum' for d in date_cols
}).reset_index()
world_deaths_agg_df = world_deaths_df.groupby(['Country/Region']).agg({
    d: 'sum' for d in date_cols
}).reset_index()

# Start populating combined dataframe to pickle
for index, row in world_confirmed_agg_df.iterrows():
    country = row['Country/Region']
    population = world_population_df[world_population_df['country'] == row['Country/Region']]['Population']
    if len(population) != 1:
        print('skipping country', country, population)
        continue
    population = population.to_numpy()[0]
    for (date_col, date) in zip(date_cols, dates):
        confirmed = row[date_col]
        if confirmed == 0:
            continue
        if date_col in world_deaths_agg_df:
            deaths = world_deaths_agg_df[
                (world_deaths_agg_df['Country/Region'] == row['Country/Region'])
            ][date_col].to_numpy()[0]
        else:
            deaths = 0
            
        values = [country, date, confirmed, deaths, population]
        df_length = len(world_combined_df)
        world_combined_df.loc[df_length] = values
    if index % 100 == 0:
        print('processed {} out of {}'.format(index, len(world_confirmed_agg_df)))
world_combined_df.tail()   

In [ ]:
with open('./data/world_combined_df.pkl', 'wb') as f:
    pickle.dump(world_combined_df, f)